# Test Processing of a single cycling data

The cycling data will include both RPTs and information during cycling. First, split up these two pieces. Then, for each piece, parse the data.

Compile information on UMBLFEB2022 formation cells

1/17/2023

In [1]:
import os, sys
from matplotlib import pyplot as plt

if os.path.basename(os.getcwd()) == 'notebooks-2023':
    os.chdir('../')
    sys.path.insert(0, 'src/')
    
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib ipympl

import src.plotter as plotter
import src.vas as vas
import src.parsers as parsers

%load_ext autoreload
%autoreload 2

plotter.initialize(plt)

In [2]:
# Initialize the helper class
vh = vas.VasHelper()

Initializing Voltaiq Analytic Studio Helper...
Initializing test records...
Initializing devices...
Done.


In [5]:
device_name = 'UMBL2022FEB_CELL152030'
cyc = parsers.CyclingDataParser(device_name, vh)

Working on "UMBL2022FEB_CELL152030_CYC_1C1CR1_P45C_5P0PSI_20220804_R1"...
Initialization complete.


In [6]:
rpt = parsers.RptDataParser(cyc.df_rpt)

# First inspect the entire dataset

In [7]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(8, nrows * 3), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(cyc.df[xvar], cyc.df[yvar])
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(12, nrows * 3), sharex=True)


for yvar, ax in zip(plot_vars, axs):
    ax.plot(cyc.df_rpt[xvar], cyc.df_rpt[yvar], ls='', marker='o', ms=0.3, c='r')
    ax.plot(cyc.df_cyc[xvar], cyc.df_cyc[yvar], ls='', marker='o', ms=0.3, c='b')
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);
axs[0].set_ylim((2.7, 4.22))
axs[0].set_title(device_name);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
df_by_cyc = cyc.get_cycling_info()
df_by_cyc

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,discharge_energy_wh,tot_charge_capacity_ah,tot_charge_energy_wh,tot_discharge_capacity_ah,tot_discharge_energy_wh,charge_cc_time_s,charge_cv_time_s,charge_rest_delta_voltage_v,discharge_cc_time_s,discharge_rest_delta_voltage_v
cycle_index,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.000000,0.000000,0.161482,0.511116,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.081936,0.268059,0.168439,0.531223,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.163858,0.548809,0.175382,0.553034,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.245795,0.835924,0.182333,0.575806,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0.327736,1.124766,0.189288,0.598931,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,NaN,NaN,NaN,NaN,2286.403605,9194.775027,2284.192846,7860.127661,NaN,NaN,NaN,NaN,NaN
1612,NaN,NaN,NaN,NaN,2287.038497,9197.439459,2284.199794,7860.150892,NaN,NaN,NaN,NaN,NaN
1613,NaN,NaN,NaN,NaN,2288.937512,9204.618198,2286.127168,7867.248592,NaN,NaN,NaN,NaN,-0.000162


In [13]:
plot_vars = ['discharge_capacity_ah', 
             'charge_cc_time_s', 
             'charge_cv_time_s', 
             'charge_rest_delta_voltage_v',
             'discharge_cc_time_s', 
             'discharge_rest_delta_voltage_v']

xvar = 'tot_discharge_capacity_ah'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(12, nrows * 4), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(df_by_cyc[xvar], df_by_cyc[yvar])
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    if ax == axs[0]:
        ax.set_title(f'{device_name} \n {yvar}')
    else:
        ax.set_title(yvar)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process the RPT Data

In [30]:
rpt_start_cycle_list = cyc.get_rpt_start_cycles()
df_by_rpt = rpt.get_rpt_info(rpt_start_cycle_list)

# Append the throughput data by matching on the cycling info DataFrame
rpt_tot_discharge_capacity_ah = []

for cycle_index in df_by_rpt['cycle_index']:
    rpt_tot_discharge_capacity_ah.append(df_by_cyc['tot_discharge_capacity_ah'][df_by_cyc.index == cycle_index].values[0])
    
df_by_rpt['tot_discharge_capacity_ah'] = rpt_tot_discharge_capacity_ah

df_by_rpt

,cycle_index,c20_discharge_capacity_ah,c20_charge_capacity_ah,deltav_4p2v_12hr,deltav_3p0v_2p5hr,dvdq_data,tot_discharge_capacity_ah
0,1,2.434554,2.440033,-0.009569,1.060504,"{'dch_q': [-1.8252372979444718e-07, 0.00034686...",0.161482
1,85,2.369527,2.371714,-0.008920,0.974549,"{'dch_q': [-5.372206119034492e-06, 0.000342161...",118.642797
2,166,2.327987,2.328732,-0.008596,0.952816,"{'dch_q': [7.938370890106361e-07, 0.0003480087...",231.504682
3,296,2.272575,2.271100,-0.008920,0.935787,"{'dch_q': [-1.928014392783047e-06, 0.000345371...",451.367368
4,425,2.223601,2.219960,-0.009569,0.914866,"{'dch_q': [8.854807631538997e-07, 0.0003481888...",664.225729
5,553,2.190510,2.177586,-0.010218,0.924759,"{'dch_q': [1.2310301084160235e-06, 0.000349036...",871.426266
6,681,2.146436,2.129819,-0.011191,0.901080,"{'dch_q': [-4.679087879675073e-08, 0.000348128...",1072.547781
7,808,2.103083,2.093045,-0.011353,0.877888,"{'dch_q': [1.1506935834515585e-06, 0.000348517...",1267.720325
8,934,2.059033,2.050123,-0.012650,0.851615,"{'dch_q': [1.74864180634648e-06, 0.00034899725...",1455.841153
9,1059,2.039194,2.016600,-0.013136,0.822260,"{'dch_q': [1.3777586286774244e-06, 0.000349502...",1635.342729


In [31]:
fh2, axs2 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh3, axs3 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_by_rpt)+1))))

i = 0
for cycle_index, row in df_by_rpt.iterrows():
    
    data = row['dvdq_data'];
    
    axs2[0].plot(data['chg_q'], data['chg_v'], c=colors[i])
    axs2[1].plot(data['dch_q'], data['dch_v'], c=colors[i])
    
    axs3[0].plot(data['chg_q'], data['chg_dvdq'], c=colors[i])
    axs3[1].plot(data['dch_q'], data['dch_dvdq'], c=colors[i])

    axs3[0].set_ylim([0, 0.75])
    axs3[1].set_ylim([-0.75, 0])
                 
    i += 1
    
axs2[0].set_title(device_name)
axs2[1].set_xlabel('Capacity (Ah)')
axs2[0].set_ylabel('Voltage (V)')
axs2[1].set_ylabel('Voltage (V)')
axs3[0].set_title(device_name)
axs3[1].set_xlabel('Capacity (Ah)')
axs3[0].set_ylabel('dV/dQ (V/Ah)')
axs3[1].set_ylabel('dV/dQ (V/Ah)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plotting Cycling and RPT data together

In [32]:
plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['c20_discharge_capacity_ah'], marker='o', label='C/20 DCH')
plt.plot(df_by_cyc['tot_discharge_capacity_ah']/2.5, df_by_cyc['discharge_capacity_ah'], label='1C DCH')
plt.legend()
plt.ylabel('Capacity (Ah)')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name);

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_3p0v_2p5hr'], marker='o', label='$\Delta$V, 3.0V, 2.5hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name);

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_4p2v_12hr'], marker='o', label='$\Delta$V, 4.2V, 12hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah)')
plt.grid(False)
plt.title(device_name);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process HPPC data from each RPT

In [39]:
df_hppc = rpt.get_hppc_info(rpt_start_cycle_list)

# Append the throughput data by matching on the cycling info DataFrame
hppc_tot_discharge_capacity_ah = []

for cycle_index in df_hppc['cycle_index']:
    hppc_tot_discharge_capacity_ah.append(df_by_cyc['tot_discharge_capacity_ah'][df_by_cyc.index == cycle_index].values[0])
    
df_hppc['tot_discharge_capacity_ah'] = hppc_tot_discharge_capacity_ah

df_hppc

,cycle_index,resistance_discharge_ohms,resistance_charge_ohms,capacity_ah,tot_discharge_capacity_ah
0,1,0.086214,0.071357,0.000000,0.161482
1,1,0.065927,0.059111,0.074974,0.161482
2,1,0.057287,0.053531,0.149953,0.161482
3,1,0.050547,0.048927,0.224938,0.161482
4,1,0.048276,0.047172,0.299907,0.161482
...,...,...,...,...,...
365,1593,0.212496,0.187780,1.049688,2284.073335
366,1593,0.212175,0.186858,1.124669,2284.073335
367,1593,0.211593,0.186080,1.199650,2284.073335
368,1593,0.211013,NaN,1.274631,2284.073335


In [40]:
plt.figure()

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_hppc['cycle_index'].unique())+1))))

for idx, cycle_index in enumerate(reversed(df_hppc['cycle_index'].unique())):
    
    curr_df = df_hppc[df_hppc['cycle_index'] == cycle_index]
    curr_throughput_ah = curr_df['tot_discharge_capacity_ah'].head(1).values[0]
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_discharge_ohms'], label=f'{curr_throughput_ah:.0f} Ah', marker='o', color=colors[idx])
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_charge_ohms'], label=f'', marker='x', linestyle='', color=colors[idx])
       
plt.legend()
plt.xlabel('Capacity (Ah)')
plt.ylabel('10s Resistance ($\Omega$)')
plt.title(device_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'UMBL2022FEB_CELL152030')